In [10]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from omegaconf import OmegaConf
import numpy as np

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [11]:
def load_all_experiments(experiments_root="outputs/experiments"):
    """Load all experiments from the outputs directory"""
    experiments_path = Path(experiments_root)
    experiments = []
    
    if not experiments_path.exists():
        print(f"Directory {experiments_root} doesn't exist")
        return experiments
    
    for exp_name_dir in experiments_path.iterdir():
        if not exp_name_dir.is_dir():
            continue
            
        print(f"Scanning experiment: {exp_name_dir.name}")
        
        for timestamp_dir in exp_name_dir.iterdir():
            if not timestamp_dir.is_dir():
                continue
                
            # Check for Hydra config (always exists) and our loss file
            config_file = timestamp_dir / ".hydra" / "config.yaml"
            loss_file = timestamp_dir / "loss_summary.txt"
            
            if config_file.exists():  # Hydra config should always exist
                try:
                    # Load config
                    config = OmegaConf.load(config_file)
                    
                    # Load losses if available
                    best_loss = float('inf')
                    final_loss = float('inf')
                    
                    if loss_file.exists():
                        with open(loss_file, 'r') as f:
                            for line in f:
                                if line.startswith('best_loss='):
                                    best_loss = float(line.split('=')[1].strip())
                                elif line.startswith('final_loss='):
                                    final_loss = float(line.split('=')[1].strip())
                    
                    # Create summary from config and loss data
                    summary = {
                        'experiment_name': config.experiment.name,
                        'best_loss': best_loss,
                        'final_loss': final_loss,
                        'has_loss_data': loss_file.exists(),
                        'total_steps': config.training.max_steps,
                        'architecture': {
                            'potential_layers': config.arch.potential.hidden_layers,
                            'potential_size': config.arch.potential.layer_size,
                            'cv_layers': config.arch.CV.hidden_layers,
                            'cv_size': config.arch.CV.layer_size,
                        }
                    }
                    
                    experiment = {
                        'name': exp_name_dir.name,
                        'timestamp': timestamp_dir.name,
                        'path': str(timestamp_dir),
                        'config': config,
                        'summary': summary
                    }
                    
                    experiments.append(experiment)
                    if loss_file.exists():
                        print(f"  ✅ Loaded run: {timestamp_dir.name}")
                    else:
                        print(f"  📋 Loaded run: {timestamp_dir.name} (config only)")
                        
                except Exception as e:
                    print(f"  ❌ Error loading {timestamp_dir.name}: {e}")
    
    print(f"\nTotal experiments loaded: {len(experiments)}")
    return experiments

In [12]:
# Cell 3: Load Your Experiments
experiments = load_all_experiments()

# Display what we found
for exp in experiments[-5:]:  # Show last 5
    print(f"{exp['name']} ({exp['timestamp']}) - Best Loss: {exp['summary']['best_loss']:.2e}")

Scanning experiment: base_reduced_ntk_scheduler_tests
  📋 Loaded run: 2025-06-20_14-56-19 (config only)
  📋 Loaded run: 2025-06-20_14-38-55 (config only)
  📋 Loaded run: 2025-06-20_14-44-04 (config only)
Scanning experiment: nexPINNACLE_reduced_interior_sampling
  📋 Loaded run: 2025-06-18_15-07-00 (config only)
Scanning experiment: base_reduced_ntk_activation_tests
  📋 Loaded run: 2025-06-19_17-53-59 (config only)
  📋 Loaded run: 2025-06-19_17-28-33 (config only)
  📋 Loaded run: 2025-06-19_17-41-07 (config only)
  📋 Loaded run: 2025-06-19_16-58-07 (config only)
  📋 Loaded run: 2025-06-19_17-13-46 (config only)
  📋 Loaded run: 2025-06-19_17-19-47 (config only)
  📋 Loaded run: 2025-06-19_16-54-18 (config only)
Scanning experiment: base+ntk
  📋 Loaded run: 2025-06-18_14-54-06 (config only)
  📋 Loaded run: 2025-06-18_14-16-30 (config only)
  📋 Loaded run: 2025-06-18_13-44-25 (config only)
  📋 Loaded run: 2025-06-18_13-44-45 (config only)
  📋 Loaded run: 2025-06-18_13-43-34 (config only)
  